# GalaxyChop's Tutorial

This tutorial is a guide to use GalaxyChop package. GalaxyChop performs the dynamical descomposition of a galaxy with tree diferent methods.

In this tutorial we show you step by step how to get started with the GalaxyChop package.

---------------------------------------------------------------------------------------------

## Import GalaxyChop and the necesary packages

In [1]:
import numpy as np
import astropy.units as u
import galaxychop as gc

The first thing to do is to import the necessary packages:

* `numpy`: it is used to load the data.
* `astropy.units`: it is used to introduce the units to our data.
* `galaxychop`: it is the library that we present in this tutorial.

## Loading the data

For this tutorial we use a preloaded dataset offered by GalaxyChop

In [2]:
star = gc.dataset.load_star_394242()
dark = gc.dataset.load_gas_394242()
gas = gc.dataset.load_dark_394242()
pot_star = gc.dataset.load_pot_star_394242()
pot_dark = gc.dataset.load_pot_dark_394242()
pot_gas = gc.dataset.load_pot_gas_394242()

The first step is to load mass (m), positions (x, y, z) and velocities (vx, vy, vz) of the galaxy.  The data must be loaded in the order: stars, dark matter and gas. If the user previously has calculated the potential energy (pot) of all partcicles (stars, dark matter and gas), it also can be introduced.

The data **must be entered with units**. Masses in ${\rm M_{\odot}}$, positions in ${\rm kpc}$, velocities in ${\rm km/s}$ and potential energy in ${\rm (km/s)^2}$. (Note that the potential energy we use here is an specific potential energy.). For this we use the package `units` from `astropy`

In [3]:
galaxy = gc.Galaxy(
    m_s = star[:,0]*u.M_sun,
    x_s = star[:,1]*u.kpc,
    y_s = star[:,2]*u.kpc, 
    z_s = star[:,3]*u.kpc,
    vx_s = star[:,4]*u.km / u.s,
    vy_s = star[:,5]*u.km / u.s,
    vz_s = star[:,6]*u.km / u.s,
    m_dm = dark[:,0]*u.M_sun,
    x_dm = dark[:,1]*u.kpc,
    y_dm = dark[:,2]*u.kpc,
    z_dm = dark[:,3]*u.kpc,
    vx_dm = dark[:,4]*u.km / u.s,
    vy_dm = dark[:,5]*u.km / u.s,
    vz_dm = dark[:,6]*u.km / u.s,
    m_g = gas[:,0]*u.M_sun,
    x_g = gas[:,1]*u.kpc,
    y_g = gas[:,2]*u.kpc,
    z_g = gas[:,3]*u.kpc,
    vx_g = gas[:,4]*u.km / u.s,
    vy_g = gas[:,5]*u.km / u.s,
    vz_g = gas[:,6]*u.km / u.s,
    pot_s = pot_star*(u.km / u.s)**2,
    pot_dm = pot_dark*(u.km / u.s)**2,
    pot_g = pot_gas*(u.km / u.s)**2
)

## Creating the galaxy

**galaxy** is now an instantiated object with the Galaxy class.

There are a number of galaxy properties that we can calculate if necessary

* **Specific kinetical energy**
* **Specific potential energy**
* **Specific total energy**
* **Specific angular momentum**
* Pairs of points (x,y) used for interpolation of the **Circular Angular Moment** function
* **Circularity parameter**: in the z-axis and in the plane.

**Note**

In case it is not necessary to work or analyze any of the parameters mentioned above you can directly perform the dynamical decomposition. For this, you can proceed directly to the title **Dynamic decomposition methods**.

#### **Specific energy**

In [4]:
E_s, E_dm, E_g = galaxy.energy

This returns the total specific energy of the components of stars (s), dark matter (dm) and gas (g). 
If you prefer separately specific kinetic and/or potential energy, you can run:


In [5]:
K_s, K_dm, K_g = galaxy.kinetic_energy

for kinetic energy, or:

In [6]:
U_s = galaxy.potential_energy().pot_s
U_dm = galaxy.potential_energy().pot_dm
U_g = galaxy.potential_energy().pot_g

ValueError: Potentials are already calculated

for potential energy. 

*You must note that `kinetic_energy` is a property of the class `Galaxy`, and `potential_energy` is a method. This is impremented in that way to allow entering potentials manually if you prefer.*

#### Specific angular momentum

In [7]:
J_part = galaxy.angular_momentum().J_part
J_star = galaxy.angular_momentum().J_star
Jr_part = galaxy.angular_momentum().Jr_part
Jr_star = galaxy.angular_momentum().Jr_star

This returns the parameters:

* **J_part**: Specific angular momentum of stars, dark matter and gas.
* **J_star**: Specific angular momentum for stars.
* **Jr_part**: Absolute value of the specific angular momentum in the XY plane of all particles (stars, dark matter and gas).
* **Jr_star**: Absolute value of the specific angular momentum in the XY plane of star particles.

#### Circular angular momentum

In [8]:
x = galaxy.jcirc().x
y = galaxy.jcirc().y

This returns a new galaxy with the parameters:

* **x**: Stellar normalized energy.
* **y**: z component of the normalized angular momentum.

For more information you can access the documentation of the jcirc method in the Notes section

#### Circularity parameters

In [9]:
E_star, eps, eps_r = galaxy.paramcirc

This returns the **normalized specific stellar energy**, **circularity parameter in z** and **circularity parameter in plane**.

## Dynamic decomposition methods

In GalaxyChop we present 4 methods to perform the dynamical decomposition of galaxies stellar component.

* **GCAbadi**
* **GCChop**
* **GCgmm**
* **GCAutogmm**
* **GCKmeans**

#### GCAbadi

This is an implementation of the method for dynamically decomposing galaxies described by [Abadi et al.(2003)](https://ui.adsabs.harvard.edu/abs/2003ApJ...597...21Aabstract).

First, we instantiate an object with the class `GCAbadi`.

In [10]:
gcabadi = gc.GCAbadi()

Next, we decompose the previously instantiated galaxy

In [11]:
gcabadi.decompose(galaxy)

The previous method certifies that what we introduce an instantiated galaxy with the `Galaxy` class, and performs the decomposition.

Then, we obtain the different groups to which each particle of the galaxy belongs: 

In [12]:
gcabadi.labels_

array([-1, -1,  0, ...,  0,  0,  0])

#### GCChop

This is an implementation of the method for dynamically decomposing galaxies used in [Tissera et al.(2012)](https://ui.adsabs.harvard.edu/abs/2012MNRAS.420..255T/abstract), [Vogelsberger et al.(2014)](https://ui.adsabs.harvard.edu/abs/2014MNRAS.444.1518V/abstract), [Marinacci et al.(2014)](https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.1750M/abstract), [Park et al.(2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...883...25P/abstract), etc.

We instantiate an object with the class `GCChop`.

In [13]:
gcchop = gc.GCChop()

Next, we decompose the previously instantiated galaxy

In [14]:
gcchop.decompose(galaxy)

The previous method certifies that what we introduce an instantiated galaxy with the `Galaxy` class, and performs the decomposition.

Then, we obtain the different groups to which each particle of the galaxy belongs: 

In [15]:
gcchop.labels_

array([-1, -1,  0, ...,  0,  0,  1])

#### GCGmm

This is an implementation of the method for dynamically decomposing galaxies described by [Obreja et al.(2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.487.4424O/abstract)

First we instantiate an object with the class `GCgmm`.

In [16]:
gcgmm = gc.GCGmm(n_components=3)

Next, we decompose the previously instantiated galaxy

In [17]:
gcgmm.decompose(galaxy)

GCGmm()

The previous method certifies that what we introduce an instantiated galaxy with the `Galaxy` class, and performs the decomposition.

Then, we obtain the different groups to which each particle of the galaxy belongs: 

In [18]:
gcgmm.labels_

array([-1, -1,  2, ...,  0,  1,  0])

#### GCAutogmm

This is an implementation of the method for dynamically decomposing galaxies described by [Du et al.(2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...884..129D/abstract)

First we instantiate an object with the class `GCAutogmm`.

In [19]:
gcautogmm = gc.GCAutogmm()

Next, we decompose the previously instantiated galaxy

In [20]:
gcautogmm.decompose(galaxy)

The previous method certifies that what we introduce an instantiated galaxy with the `Galaxy` class, and performs the decomposition.

Then, we obtain the different groups to which each particle of the galaxy belongs: 

In [21]:
gcautogmm.labels_

array([-1, -1,  1, ...,  0,  0,  3])

#### GCKmeans

This is an implementation of [Skitlearn](https://scikit-learn.org/stable/about.html#citing-scikit-learn) K-means as a method for dynamically decomposing galaxies. 

First, we instantiate an object with the class `GCKmeans` by translating the amount of components that we want to obtain from the decomposition.

In [22]:
gckmeans = gc.GCKmeans(n_clusters=3)

Then, we decompose the previously instantiated galaxy.

In [23]:
gckmeans.decompose(galaxy)

GCKmeans()

This method certifies that what we introduce an instantiated galaxy with the `Galaxy` class, and performs the decomposition.

Then, we obtain the different groups to which each particle of the galaxy belongs runing:

In [24]:
gckmeans.labels_

array([-1, -1,  1, ...,  0,  1,  0])